In [2]:
import sys
import os
import numpy as np
import pandas as pd
from IPython.utils import io
#Adjust directories to own paths
current_dir = os.path.dirname(r"C:\Users\JNoot\Documents\University\Bachelor Thesis\New Code\forecasting\forecasts")
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
sys.path.insert(0, parent_dir)
sys.path.append(os.path.dirname(os.path.abspath(r"C:\Users\JNoot\Documents\University\Bachelor Thesis\New Code\forecasting\forecasts\perform_forecasts.ipynb")) + '/..')


from forecasting.data_preprocessing.obtain_data import obtainData
from forecasting.forecasts.in_sample_forecasts import in_sample_forecast
from forecasting.forecasts.out_of_sample_forecasts import forecast
from forecasting.utils.squared_errors import get_errors
from forecasting.utils.qlike import get_qlike
from forecasting.utils.model_confidence_set import update_mcs_count
from forecasting.utils.market_cycles import define_market_phases, cycle_errors, segment_data_by_phases, perform_kruskal, perform_dunn_test
from forecasting.utils.large_mid_cap import get_grouped_errors, initialize_errors, collect_errors, compute_statistics
from forecasting.utils.utility_benefits import expected_utility
from scipy.stats import skew
from tqdm import notebook


def perform_models(coins, in_sample, training_size=0.7):
    results = {
        'coin': [],
        'LLF_rmse': [], 'RF_rmse': [], 'GARCH_rmse': [], 'GJR_rmse': [], 'HAR-RV_rmse': [],
        'LLF_mae': [], 'RF_mae': [], 'GARCH_mae': [], 'GJR_mae': [], 'HAR-RV_mae': [],
        'LLF_qlike': [], 'RF_qlike': [], 'GARCH_qlike': [], 'GJR_qlike': [], 'HAR-RV_qlike': [],
        'LLF_utility': [], 'RF_utility': [], 'GARCH_utility': [], 'GJR_utility': [], 'HAR-RV_utility': []
    }

    mcs_counts_rmse = {model: 0 for model in ['LLF', 'RF', 'GARCH', 'GJR', 'HAR-RV']}
    mcs_counts_qlike = {model: 0 for model in ['LLF', 'RF', 'GARCH', 'GJR', 'HAR-RV']}
    mcs_counts_utility = {model: 0 for model in ['LLF', 'RF', 'GARCH', 'GJR', 'HAR-RV']}
    mcs_counts_mae = {model: 0 for model in ['LLF', 'RF', 'GARCH', 'GJR', 'HAR-RV']}
    mcsr_rmse = {}
    mcsr_qlike = {}
    mcsr_utility = {}
    mcsr_mae = {}

    # Large vs. Mid cap.
    large_cap_coins = ["Bitcoin", "Ethereum", "Tether", "Binance Coin"]
    mid_cap_coins = ["Bitcoin Cash", "Litecoin", "Internet Computer", "Polygon"]
    large_cap_errors = initialize_errors()
    mid_cap_errors = initialize_errors()

    for coin in notebook.tqdm(coins):
        print(coin)
        with io.capture_output() as captured:
            X, Y, X_ridge = obtainData(coin)
        if in_sample:
            predictions, Y_test = in_sample_forecast(X, Y, X_ridge)
        else:
            predictions, Y_test = forecast(coin, X, Y, X_ridge, training_size)
        
        mse, mae, rmse = get_errors(predictions, Y_test)
        qlike = get_qlike(predictions, Y_test)
        utility = expected_utility(predictions, Y_test)

        results['coin'].append(coin)
        for metric, res_dict in zip(['rmse', 'mae', 'qlike', 'utility'], [rmse, mae, qlike, utility]):
            for model in ['LLF', 'RF', 'GARCH', 'GJR', 'HAR-RV']:
                results[f'{model}_{metric}'].append(res_dict[model])

        mcs_counts_rmse, mcs_counts_mae, mcs_counts_qlike, mcs_counts_utility = update_mcs_count(
            predictions, Y_test, mcs_counts_rmse=mcs_counts_rmse,  mcs_counts_mae= mcs_counts_mae, mcs_counts_qlike=mcs_counts_qlike, mcs_counts_utility=mcs_counts_utility
        )

        large_cap_errors, mid_cap_errors = collect_errors(predictions, Y_test, coin, large_cap_coins, mid_cap_coins, large_cap_errors, mid_cap_errors)

        print('rmse', rmse)
        print('qlike', qlike)
        print('counts', mcs_counts_rmse, mcs_counts_mae, mcs_counts_qlike, mcs_counts_utility)
        
        if not in_sample:
            # Define market phases
            market_phases = define_market_phases(X['Close'])
            total_errors = cycle_errors(predictions, Y_test)
            errors_by_phase, phase_counts = segment_data_by_phases(total_errors, market_phases)

            # Print the number of observations in each phase
            print("Number of observations in each phase:")
            for phase, count in phase_counts.items():
                print(f"{phase}: {count}")

            if coin != 'Tether':
                # Perform Kruskal-Wallis and Dunn's tests
                kruskal_results = perform_kruskal(errors_by_phase)
                dunn_results = perform_dunn_test(errors_by_phase)

                # Print statistics, Kruskal-Wallis results, and Dunn's test results for each model
                for model, phase_data in errors_by_phase.items():
                    print(f"Statistics for model {model} on coin {coin}:")
                    for phase, errors in phase_data.items():
                        errors_array = np.array(errors)
                        print(f"{phase}:")
                        print(f"  Mean: {np.mean(errors_array)}")
                        print(f"  Standard Deviation: {np.std(errors_array)}")
                        print(f"  Skewness: {skew(errors_array)}")

                    # Print Kruskal-Wallis results
                    kruskal_result = kruskal_results[model]
                    print(f"\nKruskal-Wallis results for model {model} on coin {coin}:")
                    print(f"Statistic = {kruskal_result['stat']}, p-value = {kruskal_result['p_value']}")

                    # Print Dunn's test results
                    dunn_result = dunn_results[model]
                    print(f"\nDunn's test results for model {model} on coin {coin} (p-values):")
                    print(dunn_result)
    
    # Calculate MCSR
    for model in predictions.keys():
        mcsr_rmse[model] = mcs_counts_rmse[model] / len(coins)
        mcsr_mae[model] = mcs_counts_mae[model] / len(coins)
        mcsr_qlike[model] = mcs_counts_qlike[model] / len(coins)
        mcsr_utility[model] = mcs_counts_utility[model] / len(coins)

    results_df = pd.DataFrame(results)

    large_cap_statistics = compute_statistics(large_cap_errors)
    mid_cap_statistics = compute_statistics(mid_cap_errors)

    print("Large Cap Errors:")
    print(large_cap_statistics)
    print("\nMid Cap Errors:")
    print(mid_cap_statistics)

    # Generate and print LaTeX tables
    latex_tables = {
        'RMSE': results_df[['coin', 'LLF_rmse', 'RF_rmse', 'GARCH_rmse', 'GJR_rmse', 'HAR-RV_rmse']].to_latex(index=False, float_format="%.3f"),
        'MAE': results_df[['coin', 'LLF_mae', 'RF_mae', 'GARCH_mae', 'GJR_mae', 'HAR-RV_mae']].to_latex(index=False, float_format="%.3f"),
        'QLIKE': results_df[['coin', 'LLF_qlike', 'RF_qlike', 'GARCH_qlike', 'GJR_qlike', 'HAR-RV_qlike']].to_latex(index=False, float_format="%.3f"),
        'Utility': results_df[['coin', 'LLF_utility', 'RF_utility', 'GARCH_utility', 'GJR_utility', 'HAR-RV_utility']].to_latex(index=False, float_format="%.3f"),
        'MCSR': pd.DataFrame([mcsr_rmse, mcsr_mae, mcsr_qlike, mcsr_utility], index=['RMSE', 'MAE', 'QLIKE', 'Utility']).to_latex(float_format="%.3f")
    }
    
    #Save tables to results
    current_dir = os.getcwd()
    # Construct the path to the 'feature_importances' directory within 'results' in 'forecasting'
    tables_dir = os.path.join(current_dir, "..", "Results", "tables")

    for name, table in latex_tables.items():
        feature = name.lower()
        if in_sample:
            save_path = os.path.join(tables_dir, f"{feature}_in_sample.tex")
        else:
            save_path = os.path.join(tables_dir, f"{feature}_out-of_sample.tex")
        with open(save_path, 'w') as f:
            f.write(table)

    return latex_tables

coins = ["Bitcoin", "Ethereum", "Tether", "Binance Coin", "Bitcoin Cash", "Litecoin", "Internet Computer", "Polygon"]

#In-sample Forecasts
latex_tables_in_sample = perform_models(coins, in_sample=True)
for name, table in latex_tables_in_sample.items():
    print(f"{name} Table:")
    print(table)

# Out-of-Sample Forecasts
latex_tables_out_sample = perform_models(coins, in_sample=False, training_size=0.70)
for name, table in latex_tables_out_sample.items():
    print(f"{name} Table:")
    print(table)

  0%|          | 0/8 [00:00<?, ?it/s]

Bitcoin



  0%|          | 0/50 [00:00<?, ?it/s]

LocalLinearForest {'n_estimators': 500, 'max_depth': 9, 'min_samples_split': 8, 'min_samples_leaf': 2, 'lam': 0.1}



  0%|          | 0/50 [00:00<?, ?it/s]

...

RandomForest {'n_estimators': 400, 'max_depth': 10, 'min_samples_split': 6, 'min_samples_leaf': 5}
GARCH estimates: mu         -0.130478
omega       0.802847
alpha[1]    0.119408
beta[1]     0.858346
Name: params, dtype: float64
GJR estimates: mu         -0.085877
omega       0.849500
alpha[1]    0.143828
gamma[1]   -0.041087
beta[1]     0.853303
Name: params, dtype: float64
rmse {'LLF': 0.10596767606638481, 'RF': 1.067258658880269, 'GARCH': 4.166364671365065, 'GJR': 4.163997256791119, 'HAR-RV': 3.7247971850811714}
qlike {'LLF': 0.0006932141850227527, 'RF': 0.010937914685072018, 'GARCH': 0.5832592130640931, 'GJR': 0.5825595647181323, 'HAR-RV': 0.3126449045904578}
counts {'LLF': 7, 'RF': 1, 'GARCH': 0, 'GJR': 0, 'HAR-RV': 0} {'LLF': 6, 'RF': 2, 'GARCH': 0, 'GJR': 0, 'HAR-RV': 0} {'LLF': 5, 'RF': 4, 'GARCH': 0, 'GJR': 0, 'HAR-RV': 0} {'LLF': 5, 'RF': 4, 'GARCH': 0, 'GJR': 0, 'HAR-RV': 0}
Polygon



  0%|          | 0/50 [00:00<?, ?it/s]

LocalLinearForest {'n_estimators': 200, 'max_depth': 8, 'min_samples_split': 7, 'min_samples_leaf': 4, 'lam': 0.1}



  0%|          | 0/50 [00:00<?, ?it/s]

RandomForest {'n_estimators': 100, 'max_depth': 9, 'min_samples_split': 9, 'min_samples_leaf': 2}
GARCH estimates: mu         -0.008827
omega       0.852973
alpha[1]    0.132202
beta[1]     0.842927
Name: params, dtype: float64
GJR estimates: mu         -0.005100
omega       0.847115
alpha[1]    0.134224
gamma[1]   -0.004554
beta[1]     0.843385
Name: params, dtype: float64
rmse {'LLF': 0.38005488704928053, 'RF': 0.6558616723212058, 'GARCH': 3.4141703779394272, 'GJR': 3.4141229383044362, 'HAR-RV': 3.30327957881331}
qlike {'LLF': 0.007297256188757079, 'RF': 0.0073806709485041665, 'GARCH': 0.408734020016309, 'GJR': 0.408677398998186, 'HAR-RV': 0.3255837071643262}
counts {'LLF': 8, 'RF': 1, 'GARCH': 0, 'GJR': 0, 'HAR-RV': 0} {'LLF': 7, 'RF': 2, 'GARCH': 0, 'GJR': 0, 'HAR-RV': 0} {'LLF': 6, 'RF': 5, 'GARCH': 0, 'GJR': 0, 'HAR-RV': 0} {'LLF': 6, 'RF': 5, 'GARCH': 0, 'GJR': 0, 'HAR-RV': 0}
Large Cap Errors:
{'LLF': {'RMSLE': 0.2793840789234562, 'std': 0.35253902182056246, 'skew': 12.47841402